In [1]:
library(httr)
library(data.table)
library("jsonlite")
library("tibble")
library("tidyr")
library("dplyr")

# curl query:
# curl "https://api.openweathermap.org/data/2.5/onecall?lat=51.509865&lon=-0.118092&exclude=minutely,hourly,alerts&appid=65d4508050d5008b768b660a688651ad" | python -mjson.tool
# Turkey Fields: 40.137442, 28.383499
    
api_key <- "appid=65d4508050d5008b768b660a688651ad"
base_url <- "https://api.openweathermap.org/data/2.5/onecall?"
url_settings <- "&exclude=minutely,hourly,alerts"

#Turkey Fields:
lat <- "&lat=40.137442"
lon <- "&lon=28.383499"
#Kassow, Germany:
#lat <- "&lat=53.881579"
#lon <- "&lon=12.072763"

# complete_url variable to store complete url address
complete_url = paste(base_url,api_key,lat,lon,url_settings, sep = "", collapse = "")
print(complete_url)
    
response <- httr::GET(complete_url)

# print header of the response:
#headers(response)

# JSON automatically parsed into names list:
cont_raw <-  httr::content(response)
cont_text <- httr::content(response, "text")
cont_list <- httr::content(response, "parsed")

#typeof(response)
typeof(cont_raw)
typeof(cont_text)
typeof(cont_list)

data_raw_ugly <- jsonlite::fromJSON(cont_text)

# solution from https://www.r-bloggers.com/2018/10/converting-nested-json-to-a-tidy-data-frame-with-r/
# flatten the nested list:
data_raw <- tibble::enframe(unlist(cont_raw))
#head(data_raw,10)

# separate() turns a single character column into multiple columns
# The data from the "name" col is split at "." into 3 columns (x1,x2,x3) 
data_sep <-
  data_raw %>% tidyr::separate(name, sep = "\\.",into = c(paste0("name", 1:3)), fill = "right")
head(data_sep,60)




Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[1] "https://api.openweathermap.org/data/2.5/onecall?appid=65d4508050d5008b768b660a688651ad&lat=40.137442&lon=28.383499&exclude=minutely,hourly,alerts"


[1] "list"

[1] "character"

[1] "list"

name1,name2,name3,value
<chr>,<chr>,<chr>,<chr>
lat,NA,NA,40.1374
lon,NA,NA,28.3835
timezone,NA,NA,Europe/Istanbul
timezone_offset,NA,NA,10800
current,dt,NA,1650387197
current,sunrise,NA,1650338564
current,sunset,NA,1650386887
current,temp,NA,280.73
current,feels_like,NA,280.73


In [2]:
# Post-processing the data

data_filt <-
 data_sep %>%
  filter(
    (name1 == "daily" & name2 == "dt") |
    (name1 == "daily" & name2 == "temp" & name3 == "day") |
    (name1 == "daily" & name2 == "humidity") |
    (name1 == "daily" & name2 == "dew_point") |
    (name1 == "daily" & name2 == "wind_speed") |
    (name1 == "daily" & name2 == "uvi") |
    (name1 == "daily" & name2 == "rain")
  ) %>%
  select(name2,value)

data_location_timezone <-
 data_sep %>%
  select(name1,value) %>%
  filter(
    (name1 == "lat") |
    (name1 == "lon") |
    (name1 == "timezone")
  )

data_location_timezone
head(data_filt,15)

name1,value
<chr>,<chr>
lat,40.1374
lon,28.3835
timezone,Europe/Istanbul


name2,value
<chr>,<chr>
dt,1650362400
temp,281.04
humidity,91
dew_point,279.1
wind_speed,3.26
rain,2.38
uvi,0.75
dt,1650448800
temp,289.3


In [3]:
# add column "date" and fill the NAs with last date, delete rows "dt"
data_clean1 <-
  data_filt %>%
  mutate(date = if_else(name2 == "dt", value, NA_character_)) %>%
  fill(date, .direction ="down") %>%
  filter(name2 != "dt")

head(data_clean1,20)

name2,value,date
<chr>,<chr>,<chr>
temp,281.04,1650362400
humidity,91,1650362400
dew_point,279.1,1650362400
wind_speed,3.26,1650362400
rain,2.38,1650362400
uvi,0.75,1650362400
temp,289.3,1650448800
humidity,44,1650448800
dew_point,276.6,1650448800


In [4]:
# Convert Unix time to date, convert values to numeric
data_clean2 <-
  data_clean1 %>%
  mutate(date = as.POSIXct(as.numeric(data_clean1$date), origin="1970-01-01")) %>%
  mutate(date = as.Date(date)) %>%
  mutate(value = as.numeric(value))

head(data_clean2,7)

name2,value,date
<chr>,<dbl>,<date>
temp,281.04,2022-04-19
humidity,91.00,2022-04-19
dew_point,279.10,2022-04-19
wind_speed,3.26,2022-04-19
rain,2.38,2022-04-19
uvi,0.75,2022-04-19
temp,289.30,2022-04-20


In [6]:
# Pivot the table, using "names2" as column names, "values" as the values and "date" as Key
# Also convert NAs to null and Kelvin to Celsius
data_pivot <- 
  data_clean2 %>%
  pivot_wider(names_from = name2, values_from = value) %>%
  mutate(rain = replace_na(rain,0)) %>%
  mutate(across(c(temp,dew_point),~{.-275.15}))
  
data_pivot
data_location_timezone

date,temp,humidity,dew_point,wind_speed,rain,uvi
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2022-04-19,5.89,91,3.95,3.26,2.38,0.75
2022-04-20,14.15,44,1.45,3.16,0.00,6.33
2022-04-21,15.94,55,6.22,2.65,0.00,6.67
2022-04-22,20.01,48,8.02,2.35,0.00,7.13
2022-04-23,22.10,48,10.01,3.00,0.00,5.79
2022-04-24,20.39,52,9.51,3.50,0.00,6.00
2022-04-25,19.88,59,11.00,2.86,0.00,6.00
2022-04-26,17.57,56,8.12,3.48,0.00,6.00


name1,value
<chr>,<chr>
lat,40.1374
lon,28.3835
timezone,Europe/Istanbul
